In [2]:
import pandas as pd
import numpy as np

import folium
from folium import Marker

In [7]:
loc = pd.read_csv('./위경도.csv')

In [8]:
loc

,name,addr,lat,long,grp
0,꼬망스케익,천안 서북구 봉정로 362,36.834341,127.145798,빵소
1,듀팡과자점,천안 서북구 늘푸른5길 22,36.827329,127.129651,빵소
2,뚜쥬루돌가마점,천안 동남구 풍세로 706,36.773078,127.139533,빵소
3,못난이꽈배기,천안 동남구 사직로 10-6,36.800660,127.150044,빵소
4,몽상가인,천안 서북구 쌍용대로 85,36.805139,127.131813,빵소
...,...,...,...,...,...
58,고려산2,NaN,36.772757,127.193504,산
59,거머산1,NaN,36.734780,127.334783,산
60,거머산2,NaN,36.736335,127.333567,산
61,거머산3,NaN,36.736637,127.341182,산


In [12]:
cheonan_map = folium.Map(location=[36.80, 127.19], zoom_start=13)

In [13]:
for name, lat, lng, grp in zip(loc['name'],loc['lat'],loc['long'],loc['grp']):
    if grp =='빵소' :
        color = 'green'
        Marker(location =[lat,lng],
           popup=grp,
           tooltip=name,
           icon=folium.Icon(color=color,icon='heart')
           ).add_to(cheonan_map)
    elif grp == '폐건물':
        color = 'red'
        Marker(location =[lat,lng],
           popup=grp,
           tooltip=name,
           icon=folium.Icon(color=color,icon='home')
           ).add_to(cheonan_map)
        folium.Circle(
            location=[lat,lng],
            radius=1000,
            fill=True,
            color="#c634eb",
            fill_color="#c634eb",
            popup=grp
        ).add_to(cheonan_map)
    elif grp =='산':
        Marker(location =[lat,lng],
            popup='산',
            tooltip=name,
            icon=folium.Icon(color='purple',icon='tree-conifer')
            ).add_to(cheonan_map)
    else : 
        color = 'blue'
        Marker(location =[lat,lng],
            popup=grp,
            tooltip=name,
            icon=folium.Icon(color=color,icon='glyphicon-road')
            ).add_to(cheonan_map)
        
    # equalizer

In [15]:
cheonan_map

In [16]:
df = pd.read_csv('./features.csv', index_col=0)
df

,두정역_dist,봉명역_dist,쌍용역_dist,천안고속버스터미널_dist,천안아산역_dist,천안역 1호선_dist,꼬망스케익_dist,듀팡과자점_dist,뚜쥬루돌가마점_dist,못난이꽈배기_dist,...,오락시설(2km),숙박시설(2km),편의시설(2km),관광시설(2km),관광시설(10km),공원 (2km),공원 (10km),산 (2km),산 (5km),산 (10km)
두정동 폐공사장,1817.0,4146.0,5028.0,3206.0,7118.0,2884.0,1181.0,1520.0,8435.0,6660.0,...,14,49,150,1,15,26,171,1.0,3,8
삼룡동 폐건물,5675.0,4641.0,5089.0,4042.0,7431.0,4529.0,7273.0,6804.0,4808.0,2981.0,...,5,27,14,3,18,12,115,4.0,5,12
쌍용동 폐공장,5757.0,973.0,944.0,4866.0,3899.0,2872.0,5267.0,4355.0,4004.0,2509.0,...,11,62,64,0,16,36,163,0.0,4,10


In [30]:
df.T.style.background_gradient(axis=1)

,두정동 폐공사장,삼룡동 폐건물,쌍용동 폐공장
두정역_dist,1817.000000,5675.000000,5757.000000
봉명역_dist,4146.000000,4641.000000,973.000000
쌍용역_dist,5028.000000,5089.000000,944.000000
천안고속버스터미널_dist,3206.000000,4042.000000,4866.000000
천안아산역_dist,7118.000000,7431.000000,3899.000000
천안역 1호선_dist,2884.000000,4529.000000,2872.000000
꼬망스케익_dist,1181.000000,7273.000000,5267.000000
듀팡과자점_dist,1520.000000,6804.000000,4355.000000
뚜쥬루돌가마점_dist,8435.000000,4808.000000,4004.000000
못난이꽈배기_dist,6660.000000,2981.000000,2509.000000


In [42]:
def score(x):
    for c, v in x.items():
        if 'dist' in c:
            x[c] = 1000/v * 0.14   # km 단위로 변환 + 역수

        else:
            if c == '주차장':
                x[c] = v * 0.093    # 주차장

            elif '('  not in c:
                x[c] = v * 0.093    # 주변 대중교통 인프라 (버스 정류장, 지하철역, 주유소)
                
            elif '관광' in c:
                x[c] = v * 0.116    # 관광시설

            elif '공원' in c:
                x[c] = v * 0.116

            elif '산' in c:
                x[c] = v * 0.116

            elif '오락' in c:
                x[c] = v * 0.047

            elif '숙박' in c:
                x[c] = v * 0.047
            
            elif '편의' in c or '음식' in c:
                x[c] = v * 0  # 편의시설

    return sum(x)

df.apply(score, axis=1).sort_values(ascending=False)

음식점(2km)
편의시설(2km)
음식점(2km)
편의시설(2km)
음식점(2km)
편의시설(2km)


쌍용동 폐공장     38.461743
두정동 폐공사장    35.681048
삼룡동 폐건물     26.196582
dtype: float64